In [1]:
import sys

# bandaid fix for running this notebook in its folder on ROAR
external_folder_path = '../'
sys.path.insert(0, external_folder_path)

from util.sparkhandler import SparkHandler
from util.dataproctools import get_extracted_wet, save_rdd, load_rdd
from pyspark import StorageLevel

We use the spark handler class so we can use a unified spark session across our functions. This essentially just helps us initialize the spark session and stores the session and context within itself

In [2]:
handler = SparkHandler(available_cores=4) # look at the parameters for SparkHandler to increase max memory
ss = handler.get_spark_session()
sc = handler.get_spark_context()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 16:15:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/10 16:16:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Here we can load in our str data. Do note, `get_extracted_wet` doesn't return the sample size exactly, but an approximate amount based on the wet we use

In [3]:
rawStrData = get_extracted_wet(spark_context=sc, approx_sample_size=10000, wet_paths_file="../data/wet.paths")

Typically, processing this code, especially at large scales, takes a lot of time. In order to not recompute every time we run this, we can save this RDD and load it later.

In [4]:
save_rdd(rawStrData, "../saved_intermediates/rawStrRDD", overwrite=True) # overwrite is false by default (NOTE: the ../ basically means to go back a directory. this ensures we create the folder in the root directory instead of the examples directory)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/raameshb/GitRepos/ds410-course-proj-redo/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/raameshb/GitRepos/ds410-course-proj-redo/.venv/lib/python3.10/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/raameshb/.local/share/uv/python/cpython-3.10.18-linux-x86_64-gnu/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
25/12/10 16:16:05 ERROR SparkHadoopWriter: Aborting job job_202512101616046868149832229214752_0009.
java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1134)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait0(Promise.scala:243)
	at scal

KeyboardInterrupt: 

Loading the data is pretty straightforward with the load_rdd function

In [3]:
loaded_str_data = load_rdd(spark_context=sc, path_to_load="../saved_intermediates/rawStrRDD")

Here's what the start of a WET file looks like

In [4]:
loaded_str_data.persist(StorageLevel.MEMORY_AND_DISK) # We use persist here because I've often run into OOM errors at large data scales after triggering exection without it
firstRDDSnippet = loaded_str_data.take(1)[0][:10000] # We take the first 10000 chars of text
loaded_str_data.unpersist() # Unpersist the data to free up storage and memory
firstRDDSnippet

'WARC/1.0\r\nWARC-Type: warcinfo\r\nWARC-Date: 2025-11-21T04:54:24Z\r\nWARC-Filename: CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz\r\nWARC-Record-ID: <urn:uuid:27966d2c-0e0c-4bd3-ab9c-2e67f0e6f7eb>\r\nContent-Type: application/warc-fields\r\nContent-Length: 372\r\n\r\nSoftware-Info: ia-web-commons.3.0.1-SNAPSHOT-20251103031243\r\nExtracted-Date: Fri, 21 Nov 2025 04:54:24 GMT\r\nrobots: checked via crawler-commons 1.6-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)\r\nisPartOf: CC-MAIN-2025-47\r\noperator: Common Crawl Admin (info@commoncrawl.org)\r\ndescription: Wide crawl of the web for November 2025\r\npublisher: Common Crawl\r\n\r\n\r\n\r\nWARC/1.0\r\nWARC-Type: conversion\r\nWARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB\r\nWARC-Date: 2025-11-15T21:55:56Z\r\nWARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>\r\nWARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>\r\nWARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2G

And this is what it looks like when printed in a prettier format

In [5]:
print(firstRDDSnippet)

WARC/1.0
WARC-Type: warcinfo
WARC-Date: 2025-11-21T04:54:24Z
WARC-Filename: CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz
WARC-Record-ID: <urn:uuid:27966d2c-0e0c-4bd3-ab9c-2e67f0e6f7eb>
Content-Type: application/warc-fields
Content-Length: 372

Software-Info: ia-web-commons.3.0.1-SNAPSHOT-20251103031243
Extracted-Date: Fri, 21 Nov 2025 04:54:24 GMT
robots: checked via crawler-commons 1.6-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
isPartOf: CC-MAIN-2025-47
operator: Common Crawl Admin (info@commoncrawl.org)
description: Wide crawl of the web for November 2025
publisher: Common Crawl



WARC/1.0
WARC-Type: conversion
WARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB
WARC-Date: 2025-11-15T21:55:56Z
WARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>
WARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>
WARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2GJXBHDFZWZ
WARC-Identified-Content-Language: zho
Content-Type: text/plain


This is just to show what the types of data look like

In [6]:
type(firstRDDSnippet), type(loaded_str_data)

(str, pyspark.core.rdd.RDD)

Now a single function call to make this into a dataframe

In [7]:
from util.dataproctools import extracted_wet_to_df

In [8]:
df = extracted_wet_to_df(spark_session=ss, extracted_wet_rdd=loaded_str_data)

and if you have pandas to help visualize

In [9]:
df.toPandas()

,warc_id,target_uri,date,languages,tld,raw_content
0,<urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>,http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%8...,2025-11-15T21:55:56Z,zho,.com,\r\n安徽禾兹电子交流圈电子网_分享电子电娱最新资讯-安徽禾兹电子交流圈电子网\n安徽禾兹...
1,<urn:uuid:ca033d18-91ca-4ca3-a00e-d3bf42eced7b>,http://0skgxi9.jtznech.com/?3235018.html?xiang...,2025-11-15T21:28:16Z,zho,.com,\r\n最新版宝马棋牌/稳定版APP-体验极致娱乐，畅享无限乐趣！\n硬件\nAPP下载\n...
2,<urn:uuid:c5f758ee-2df0-4b53-836c-b705cc3d67d6>,http://100-days-of-freedom.com/2015/10/31/tag-...,2025-11-15T21:14:55Z,"deu,eng",.com,\r\nTag 117 – Geburtstag bei BMW Marmotor | 10...
3,<urn:uuid:c659d503-6d5c-4905-9eaa-11afebc09edf>,http://10hv.com/docs,2025-11-15T21:37:54Z,zho,.com,\r\n樱花动漫imo-免费VP加速器\n樱花动漫imo\n工具|时间：2025-11-16...
4,<urn:uuid:e0b888cf-22dc-4cf4-b1df-2772fae69926>,http://10lfgzt.evs888.com/?3359101.html?qingyu...,2025-11-15T21:16:39Z,"zho,eng",.com,\r\n竞彩首页专家推荐/平板兼容APP-畅享最佳娱乐体验，尽在此处！\n侨网•正文\n侨宝...
...,...,...,...,...,...,...
22070,<urn:uuid:a6366f62-e2bf-4f43-8b95-44069e0dd306>,https://zustrebisov.sk/udalosti/pozvanka-ldo-7...,2025-11-15T20:57:23Z,slk,.sk,\r\nPozvánka – LDO 7. ročník súťaže „VLASTNÁ T...
22071,<urn:uuid:5610bead-a464-46b0-bd83-e63355517e7c>,https://zwroty.smile.pl/nadaj-zwrot?domain=mxm...,2025-11-15T20:41:15Z,pol,.pl,\r\nKod do Paczkomatu InPost | | Smile\nSkorzy...
22072,<urn:uuid:18a28d69-3090-40ec-80a0-c950bca1224f>,https://zyorna.ru/catalog/item/smola-naturalna...,2025-11-15T21:59:45Z,rus,.ru,"\r\nЛадан натуральный «Dammar» (50 г), цена — ..."
22073,<urn:uuid:7c3f1f36-3a75-4586-9a94-7d022fc1c8a8>,https://zzmicky.com/vod/lgjqp/223337.html,2025-11-15T22:09:11Z,"zho,eng",.com,\r\n《731》电影片经典回顾合集-免费高清剧情片系列资源-老光棍电影院\n首页\n电影片...


In [10]:
filtered = df.filter(((df.tld == '.gov') | (df.tld == '.edu')) & (df.languages == 'eng'))

In [11]:
filtered.toPandas()

,warc_id,target_uri,date,languages,tld,raw_content
0,<urn:uuid:1d0e5d79-65f0-49a6-9423-a210cff9ebc5>,http://dendro.cnre.vt.edu/DENDROLOGY/syllabus/...,2025-11-15T20:46:04Z,eng,.edu,\r\nVirginia Tech Dendrology Fact Sheet\nVirgi...
1,<urn:uuid:70b330e4-588a-4de3-9a1b-981bb00d2be4>,http://exhibits.lib.usu.edu/items/browse?advan...,2025-11-15T21:02:31Z,eng,.edu,\r\nBrowse Items · USU Digital Exhibits\nMENU\...
2,<urn:uuid:27e47c08-1a1e-4e9f-a829-62474c341f4a>,http://fastbook.cvpa.usf.edu/fbonline/calendar...,2025-11-15T22:03:29Z,eng,.edu,"\r\nEvents Calendar - College of Design, Art &..."
3,<urn:uuid:41f8c4e4-51d6-466b-a130-ea5492215f94>,http://mirror.umd.edu/roswiki/ccny_rgbd.html?d...,2025-11-15T20:37:37Z,eng,.edu,\r\nccny_rgbd\nccny_rgbd\n[Documentation] [Tit...
4,<urn:uuid:20c0f784-e089-49be-98e8-a8eebbf03800>,http://rtw.ml.cmu.edu/rtw/kbbrowser/city:inter...,2025-11-15T20:57:54Z,eng,.edu,\r\nRead the Web :: NELL KnowledgeBase Browser...
...,...,...,...,...,...,...
271,<urn:uuid:859d0a5a-d5cd-4631-b3f1-067af348867c>,https://wwwcp.umes.edu/blog/location/hawks-cor...,2025-11-15T21:11:43Z,eng,.edu,\r\nHawk's Corner - Corner of Somerset & Princ...
272,<urn:uuid:8442ada6-f269-42e4-943c-dbbc0133cbc0>,https://yosemite.epa.gov/oa/rhc/epaadmin.nsf/F...,2025-11-15T22:25:56Z,eng,.gov,\r\nFilings| EPA Administrative Enforcement Do...
273,<urn:uuid:649bc7c8-4752-43ef-8f5b-6f315de39adb>,https://yourwater.oregon.gov/cr-detail.php?db=...,2025-11-15T21:19:49Z,eng,.gov,\r\nContact Report Details | Data Online | Ore...
274,<urn:uuid:0f077b33-fec6-4f47-bc5e-0610824c8634>,https://youthsports.rutgers.edu/courses/week/2...,2025-11-15T20:59:19Z,eng,.edu,\r\nArchives: Events - Rutgers Youth Sports Re...


In [12]:
from pyspark.ml.feature import RegexTokenizer

In [19]:
from pyspark.ml.feature import StopWordsRemover, CountVectorizer
from pyspark.ml.clustering import LDA
from pyspark.ml import Pipeline

In [23]:
# 1) Tokenize text into tokens
tokenizer = RegexTokenizer(minTokenLength=2, gaps=False, pattern=r"\b[a-zA-Z]+[\d]*(?:[-'][a-zA-Z]+[\d]*)*\b", inputCol="raw_content", outputCol="tokenized")

# 2) Remove stopwords
remover = StopWordsRemover(inputCol="tokenized", outputCol="filtered")

# 3) Convert tokens to term-frequency vectors
cv = CountVectorizer(
    inputCol="filtered",
    outputCol="features",
    vocabSize=5000,
    minDF=1    # keep terms that appear in at least 1 document
)

# 4) LDA model (k = number of topics)
lda = LDA(
    k=12,
    maxIter=10,
    featuresCol="features"
)

# 5) Build pipeline
pipeline = Pipeline(stages=[tokenizer, remover, cv, lda])

pipeline

Pipeline_55dd4c0fb7f7

In [24]:
model = pipeline.fit(filtered)

# Extract the CV and LDA sub-models
cv_model = model.stages[2]
lda_model = model.stages[3]

vocab = cv_model.vocabulary
len(vocab), vocab[:20]

(5000,
 ['volume',
  'issue',
  'search',
  'research',
  'information',
  'university',
  'data',
  'us',
  'contact',
  'new',
  'health',
  'doi',
  'may',
  'resources',
  'program',
  'home',
  'state',
  'library',
  'view',
  'project'])

In [25]:
num_words = 10

# Spark older versions require positional args
topics = lda_model.describeTopics(num_words)

topics.show(truncate=False)

# Map term indices back to words and print nicely
for row in topics.collect():

    topic_id = row.topic
    term_indices = row.termIndices
    term_weights = row.termWeights
    terms = [vocab[idx] for idx in term_indices]

    print(f"Topic {topic_id}:")
    for term, weight in zip(terms, term_weights):
        print(f"  {term:18s} {weight:.4f}")
    print()



+-----+-----------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                                |termWeights                                                                                                                                                                                                                          |
+-----+-----------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[0, 1, 1607, 168, 542, 646, 15, 2, 88, 4077]               |[0.14908059448863986, 0.145841539637499

In [2]:
#here we are just importing neccesary packages.
import matplotlib.pyplot as plt
import pandas as pd


# We allow for the use of either Wordcloud or Seaborn for visualization
try:
    from wordcloud import WordCloud
    HAS_WORDCLOUD = True
except ImportError:
    HAS_WORDCLOUD = False

try:
    import seaborn as sns
    HAS_SEABORN = True
except ImportError:
    HAS_SEABORN = False


from pyspark.ml.linalg import DenseVector, SparseVector


#here we create a function to create visualzations based on the results we get from LDA
def visualize_topics(lda_model, vocabulary, df_transformed=None, num_words=10):

    topics_df = lda_model.describeTopics(maxTermsPerTopic=num_words)
    topics = topics_df.collect()


    topic_labels = {}
    for row in topics:
        top_terms = [vocabulary[i] for i in row.termIndices[:3]]  # this is giving the top 3 words
        label = f"Topic {row.topic} — ({', '.join(top_terms)})"
        topic_labels[row.topic] = label



    # Here we are creating our first visualzation: Bar Charts.
    num_topics = len(topics)
    fig, axes = plt.subplots(1, num_topics, figsize=(5 * num_topics, 6))
    if num_topics == 1:
        axes = [axes]

    for idx, topic_row in enumerate(topics):
        term_indices = topic_row.termIndices
        term_weights = topic_row.termWeights
        words = [vocabulary[i] for i in term_indices]
        weights = [float(w) for w in term_weights]

        ax = axes[idx]
        ax.barh(words, weights)
        ax.set_title(topic_labels[topic_row.topic])
        ax.set_xlabel("Weight")
        ax.invert_yaxis() # we do this so that the highest weight is at the top.

    plt.tight_layout()
    plt.show()

    # Here we are creating our second  visualzation:Word Clouds.
    if HAS_WORDCLOUD:
        fig, axes = plt.subplots(1, num_topics, figsize=(5 * num_topics, 4))
        if num_topics == 1:
            axes = [axes]

        for idx, topic_row in enumerate(topics):
            term_indices = topic_row.termIndices
            term_weights = topic_row.termWeights
            word_freq = {vocabulary[i]: float(w) for i, w in zip(term_indices, term_weights)}

            wordcloud = WordCloud(width=425, height=325, background_color="white") \
                .generate_from_frequencies(word_freq)

            axes[idx].imshow(wordcloud, interpolation="bilinear")
            axes[idx].set_title(topic_labels[topic_row.topic])
            axes[idx].axis("off")

        plt.tight_layout()
        plt.show()

    # Here we are creating our third visualzation: Document Topic Distributions.
    if df_transformed is not None:

     # we want to extract the topic distribution column from the df which is containing the vector of probabilites
        rows = df_transformed.select("topicDistribution").collect()
        topic_probs = []
        for row in rows:
            dist = row.topicDistribution

            if isinstance(dist, (DenseVector, SparseVector)):
                dist = dist.toArray()
            topic_probs.append([float(x) for x in dist])

        df_topics = pd.DataFrame(
            topic_probs,
            columns=[topic_labels[i] for i in range(num_topics)]
        )

        fig, ax = plt.subplots(figsize=(8, 5))
        topic_means = df_topics.mean()
        ax.bar(topic_means.index, topic_means.values)
        ax.set_ylabel("Average Probability")
        ax.set_title("Average Topic Proportions")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        if HAS_SEABORN:
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.heatmap(df_topics.T, cmap="YlOrRd", cbar_kws={"label": "Probability"}, ax=ax)
            ax.set_xlabel("Document")
            ax.set_ylabel("Topic")
            ax.set_title("Document-Topic Distribution")
            plt.tight_layout()
            plt.show()



# Here we are calling the visualizations. Uncomment when you want to employ it.
# visualize_topics(lda_model, vocab, transformed, num_words=10)


In [1]:
transformed = model.transform(df)

# Show the text and its topic distribution
transformed.select("raw_content", "topicDistribution").show(truncate=False)

NameError: name 'model' is not defined

In [ ]:
from

In [ ]:
visualize_topics(lda_model, vocab, transformed, num_words=10)